# Agents

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://taseen-junaid.medium.com/self-sovereign-identity-ssi-in-blockchain-the-future-of-internet-identity-6deb75607aa")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7528b00fa980>)

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "SSI_search", "Search for information about Self Sovereign Identity (SSI). For any questions about Self Sovereign Identity (SSI), you must use this tool!!")

In [9]:
retriever_tool.name

'SSI_search'

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools=[wiki,arxiv,retriever_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/ta-seen/.local/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='SSI_search', description='Search for information about Self Sovereign Identity (SSI). For any questions about Self Sovereign Identity (SSI), you must use this tool!!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7528caa30f70>, retriever=VectorStoreRetriever(tags=['FAISS', 'O

In [13]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3")

In [22]:
from langchain import hub

instructions = """You are an assistant."""
base_promt =  hub.pull("langchain-ai/react-agent-template")
prompt = base_promt.partial(instructions=instructions)

/home/ta-seen/.local/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [23]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import create_react_agent
agent = create_react_agent(llm, tools, prompt)

In [24]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'chat_history': '', 'instructions': 'You are an assistant.', 'tools': 'wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nSSI_search: Search for information about Self Sovereign Identity (SSI). For any questions about Self Sovereign Identity (SSI), you must use this tool!!', 'tool_names': 'wikipedia,

In [25]:
agent_executor.invoke({"input": "Tell me about Self Sovereign Identity (SSI)"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: SSI_search
Action Input: "Self Sovereign Identity (SSIExistence with Total Control:Users must have an independent existence with total control of their identities. With current systems, a user identity depends on identity providers and identity providers have immense control on their user identities. With control on identity providers, they can choose to take away an identifier or they can use identifiers’ information illegally or they can also change a user identity information too.With Self Sovereign Identity (SSI), a user must have an independent existence with total control on that user hand. Other authorities do not have any power to take away an identifier or to use identifiers’ information illegally.A user may gain credentials from an issuer and the issuer can only control the credentials that are issued by that issuer where the issuer has no power on a user identity or others credentials that 

{'input': 'Tell me about Self Sovereign Identity (SSI)',
 'output': 'It seems like we already have a comprehensive overview of Self Sovereign Identity (SSI) through the SSI_search tool! The text provides insights into the concepts of existence with total control, portability, interoperability, unlinkability, and decentralized communication.'}

In [26]:
agent_executor.invoke({"input": "Tell me about the paper 2101.10187"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: 2101.10187

/home/ta-seen/.local/lib/python3.10/site-packages/langchain_community/utilities/arxiv.py:151: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2021-01-22
Title: Trinational Automated Mobility
Authors: Jonas Vogt, Niclas Wolniak, Horst Wieker
Summary: Safe, environmentally conscious and flexible, these are the central
requirements Thought: Do I need to use a tool? No
Final Answer: The paper "Trinational Automated Mobility" with ID 2101.10187 was published on January 22, 2021, by Jonas Vogt, Niclas Wolniak, and Horst Wieker. It focuses on creating safe, environmentally conscious, and flexible automated mobility systems.

> Finished chain.


{'input': 'Tell me about the paper 2101.10187',
 'output': 'The paper "Trinational Automated Mobility" with ID 2101.10187 was published on January 22, 2021, by Jonas Vogt, Niclas Wolniak, and Horst Wieker. It focuses on creating safe, environmentally conscious, and flexible automated mobility systems.'}

In [27]:
agent_executor.invoke({"input": "Tell me about Messi"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: "Lionel MessiPage: Lionel Messi
Summary: Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a foHere's the response:

```
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: "Lionel MessiPage: Lionel Messi
Summary: Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a foHere's the response:

```
Final Answer: Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Barcelona and the Argentina national team. He is widely regarded as one of the greatest players of all time.
```

> Finished chain.


{'input': 'Tell me about Messi',
 'output': 'Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Barcelona and the Argentina national team. He is widely regarded as one of the greatest players of all time.\n```'}